In [2]:
# move to source directory
%pwd
%cd ../src
%pwd
%ls

/home/yh/git/02-ieee-fraud-detection/src
AbstractTask.py*  data/             main.py*    pipeline.py*     utils/
__init__.py*      dataset.py*       models/     storage.py*      visualization/
__pycache__/      experiment.py*    mylog.py*   tests/
config/           feature_base.py*  nbtest.py*  transformer.py*
configure.py*     features/         option.py*  trash/


In [2]:
# major modules
import pandas as pd
import numpy as np

In [3]:
%ls

AbstractTask.py*  data/             main.py*    pipeline.py*     utils/
__init__.py*      dataset.py*       models/     storage.py*      visualization/
__pycache__/      experiment.py*    mylog.py*   tests/
config/           feature_base.py*  nbtest.py*  transformer.py*
configure.py*     features/         option.py*  trash/


In [4]:
import nbtest
import sys
sys.stdout.flush()

<_Logger main (INFO)>
hello world
func called
end


In [5]:
import pathlib
from configure import Config
c = Config()
c.set_parameter(config_dir=pathlib.Path('config'), config_name='config_0000', use_option=False)


In [9]:
print(str(c.runtime.VERSION))

0000


'/home/yh/git/02-ieee-fraud-detection/notebooks'